Create a Jupyter Notebook using Vertex AI Colab Enterprise, and write the code to do the following.

1.   Import the following file into a BigQuery table: gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv.
2.   Take a couple minutes to study the data.
3.   Create an external connection in BigQuery to Vertex AI.
4.   Use the CREATE MODEL statement to add Gemini to your project using the external connection you just created.
5.   Write a SQL query that uses the ML.GENERATE_TEXT function to create a weather report or warning based on the weather data in the table.
6.   Upload the Notebook to GitHub for grading.

Create the dataset.

In [9]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS challenge_3_dataset
OPTIONS(
location="us",
default_table_expiration_days=14 );

Query is running:   0%|          |

""


Create the table from the CSV.

In [7]:
%%bigquery
CREATE OR REPLACE EXTERNAL TABLE `qwiklabs-gcp-01-a7e68aa7e5b0.challenge_3_dataset.wx_data`
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv'],
  skip_leading_rows = 1,
  allow_jagged_rows = false,
  allow_quoted_newlines = false,
  field_delimiter = ',',
  max_bad_records = 10
)


Query is running:   0%|          |

""


Add Gemini model to project.

In [12]:
%%bigquery
CREATE OR REPLACE MODEL
  `qwiklabs-gcp-01-a7e68aa7e5b0.challenge_3_dataset.gemini_model` REMOTE
WITH CONNECTION DEFAULT OPTIONS (ENDPOINT = 'gemini-2.0-flash');

Query is running:   0%|          |

""


Write a SQL query that uses the ML.GENERATE_TEXT function to create a weather report or warning based on the weather data in the table.


In [23]:
%%bigquery forecast_output
CREATE OR REPLACE TABLE
`qwiklabs-gcp-01-a7e68aa7e5b0.challenge_3_dataset.forecast_output` AS (
SELECT *
FROM
ML.GENERATE_TEXT(
MODEL `challenge_3_dataset.gemini_model`,
(
   SELECT CONCAT(
      'Provide a succinct and professional sounding narrative weather forecast for the following data: ',
      date, city, state, temperature_f, wind_speed_mph, precipitation_in, barometric_pressure_inHg, humidity_percent, weather_condition) AS prompt
   FROM `qwiklabs-gcp-01-a7e68aa7e5b0.challenge_3_dataset.wx_data`
),
STRUCT(
   0.2 AS temperature, TRUE AS flatten_json_output, 1000 AS max_output_tokens)));

Query is running:   0%|          |

Example output:

Cloudy skies are expected in Chicago today. The temperature is currently 53 degrees Fahrenheit with a wind chill of 51. The wind is blowing from the west at 5.4 miles per hour. The barometric pressure is 29.76 inches and falling.